In [11]:
#importing nltk(Natural Language Toolkit) and numpy libraries for text processing.
import nltk
import numpy as np

In [12]:
nltk.download('punkt')  #downloading model to tokenize message.
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [13]:
#stop words removal
nltk.download('stopwords')     #downloading stopwords
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [14]:
#Lemmatization (Converting a word to it's base form) eg: 'Moving'-> Lemmatization ->'Move'
#Downloading all the lemmas present in English language and WordNetLemmatizer for Lemmatization
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [15]:
#We can define our own function for stopwords removal, tokenization & lemmatization. It takes the corpus of words as an argument
def clean_corpus(corpus):
    #lowering every word in text
    corpus = [doc.lower() for doc in corpus]
    cleaned_corpus = []
    stop_words = stopwords.words('english')
    wordnet_lemmatizer = WordNetLemmatizer()
    
    #iterating over every text
    for doc in corpus:
        #tokenizing text
        tokens = word_tokenize(doc)
        cleaned_sentence = []
        for token in tokens:
            #removing stopwords and punctuations
            if token not in stop_words and token.isalpha():
                #applying lemmatization
                cleaned_sentence.append(wordnet_lemmatizer.lemmatize(token))
        cleaned_corpus.append(' '.join(cleaned_sentence))
    return cleaned_corpus
        

In [16]:
#Intent Classification
#loading the intents from 'intents.json' file.


In [17]:
import json
with open('/intents.json') as file:
  intents = json.load(file)

In [18]:
corpus = []
tags = []

for intent in intents['intents']:
    # taking all patterns in intents to train a neural network
    for pattern in intent['patterns']:
        corpus.append(pattern)
        tags.append(intent['tag'])

In [19]:
cleaned_corpus = clean_corpus(corpus)
cleaned_corpus

['hi',
 'anyone',
 'hey',
 'hola',
 'hello',
 'good day',
 'bye',
 'see later',
 'goodbye',
 'nice chatting bye',
 'till next time',
 '',
 'thanks',
 'thank',
 'helpful',
 'awesome thanks',
 'thanks helping',
 'could help',
 'help provide',
 'helpful',
 'support offered',
 'please check order status',
 'able check order status',
 'help order status',
 'order status',
 'order',
 'food',
 'track order',
 'track food',
 'hi want cancel order',
 'want cancel order',
 'please cancel order',
 'cancel order',
 'want add delivery instruction',
 'please add delivery instruction',
 'include delivery instruction',
 'tell joke',
 'feeling bored',
 'joke please',
 'make laugh',
 'want laugh']

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(cleaned_corpus)

In [21]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
y = encoder.fit_transform(np.array(tags).reshape(-1,1))

In [22]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential([
                    Dense(128, input_shape=(X.shape[1],), activation='relu'),
                    Dropout(0.2),
                    Dense(64, activation='relu'),
                    Dropout(0.2),
                    Dense(y.shape[1], activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               5888      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 520       
Total params: 14,664
Trainable params: 14,664
Non-trainable params: 0
_________________________________________________________________


In [23]:
history = model.fit(X.toarray(), y.toarray(), epochs=20, batch_size=1)

Epoch 1/20
41/41 [==============================] - 1s 2ms/step - loss: 2.0744 - accuracy: 0.1463
Epoch 2/20
41/41 [==============================] - 0s 2ms/step - loss: 1.9835 - accuracy: 0.5122
Epoch 3/20
41/41 [==============================] - 0s 2ms/step - loss: 1.8735 - accuracy: 0.5854
Epoch 4/20
41/41 [==============================] - 0s 2ms/step - loss: 1.7994 - accuracy: 0.6585
Epoch 5/20
41/41 [==============================] - 0s 2ms/step - loss: 1.6045 - accuracy: 0.8049
Epoch 6/20
41/41 [==============================] - 0s 2ms/step - loss: 1.4142 - accuracy: 0.7805
Epoch 7/20
41/41 [==============================] - 0s 2ms/step - loss: 1.1345 - accuracy: 0.8293
Epoch 8/20
41/41 [==============================] - 0s 2ms/step - loss: 0.9838 - accuracy: 0.8293
Epoch 9/20
41/41 [==============================] - 0s 2ms/step - loss: 0.8120 - accuracy: 0.9268
Epoch 10/20
41/41 [==============================] - 0s 2ms/step - loss: 0.5811 - accuracy: 0.9512
Epoch 11/20
41/41 [

In [24]:
# if prediction for every tag is low, then we want to classify that message as noanswer

INTENT_NOT_FOUND_THRESHOLD = 0.40

def predict_intent_tag(message):
  message = clean_corpus([message])
  X_test = vectorizer.transform(message)
  y = model.predict(X_test.toarray())
  
  # if probability of all intent is low, classify it as noanswer
  if y.max() < INTENT_NOT_FOUND_THRESHOLD:
    return 'noanswer'
  
  prediction = np.zeros_like(y[0])
  prediction[y.argmax()] = 1
  tag = encoder.inverse_transform([prediction])[0][0]
  return tag

print(predict_intent_tag('How you could help me?'))
print(predict_intent_tag('swiggy chat bot'))
print(predict_intent_tag('Where\'s my order'))


options
goodbye
order-status-request


In [25]:
import random
import time 

In [26]:
def get_intent(tag):
  # to return complete intent from intent tag
  for intent in intents['intents']:
    if intent['tag'] == tag:
      return intent

In [27]:
def perform_action(action_code, intent):
  # funition to perform an action which is required by intent
  
  if action_code == 'CHECK_ORDER_STATUS':
    print('\n Checking database \n')
    time.sleep(2)
    order_status = ['in kitchen', 'with delivery executive']
    delivery_time = []
    return {'intent-tag':intent['next-intent-tag'][0],
            'order_status': random.choice(order_status),
            'delivery_time': random.randint(10, 30)}
  
  elif action_code == 'ORDER_CANCEL_CONFIRMATION':
    ch = input('BOT: Do you want to continue (Y/n) ?')
    if ch == 'y' or ch == 'Y':
      choice = 0
    else:
      choice = 1
    return {'intent-tag':intent['next-intent-tag'][choice]}
  
  elif action_code == 'ADD_DELIVERY_INSTRUCTIONS':
    instructions = input('Your Instructions: ')
    return {'intent-tag':intent['next-intent-tag'][0]}

In [32]:
while True:
  # get message from user
  message = input('You: ')
  # predict intent tag using trained neural network
  tag = predict_intent_tag(message)
  # get complete intent from intent tag
  intent = get_intent(tag)
  # generate random response from intent
  response = random.choice(intent['responses'])
  print('Bot: ', response)

  # check if there's a need to perform some action
  if 'action' in intent.keys():
    action_code = intent['action']
    # perform action
    data = perform_action(action_code, intent)
    # get follow up intent after performing action
    followup_intent = get_intent(data['intent-tag'])
    # generate random response from follow up intent
    response = random.choice(followup_intent['responses'])

    # print randomly selected response
    if len(data.keys()) > 1:
      print('Bot: ', response.format(**data))
    else:
      print('Bot: ', response)

  # break loop if intent was goodb
  if tag == 'goodbye':
    break

You: Hi
Bot:  Good to see you again
You: i don't want to cancel my order
Bot:  I can cancel your order, there can be a cancellation fee depending on your order status. I need your final confirmation to confirm your order.
BOT: Do you want to continue (Y/n) ?y
Bot:  Your order is canceled, you will receive refund for this order in 2 days. Is there anything else, we can help you with?
You: i want to cancel my order
Bot:  I can cancel your order, If your order is in kitchen, there will be a cancellation fee.  Do you want me to proceed?
BOT: Do you want to continue (Y/n) ?n
Bot:  Alright, I am not canceling your order. Is there anything else, we can help you with?
You: no thankyou
Bot:  See you!
